In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.neighbors import NearestNeighbors

2021-12-11 17:02:59.985102: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-11 17:02:59.985140: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv('data/data.csv')
df = df.replace(r'\s', '', regex=True)

texts = df.ProductName.to_list()
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer.fit_on_texts(texts)

rev_word_index = {v: k for k, v in tokenizer.word_index.items()}

In [4]:
import pickle

with open('tokenizer.pickle', 'rb') as f:
    tokenizer = pickle.load(f)

tokenizer.word_index

{'пиво': 1,
 'конфеты': 2,
 'лимон': 3,
 'пюре': 4,
 'авокадо': 5,
 'картофельфри': 6,
 'грибыгруздисолёные': 7,
 'опята': 8,
 'мороженое': 9,
 'крабовыепалочки': 10,
 'хлебцы': 11,
 'пицца': 12,
 'цыплёноктабака': 13,
 'мандарины': 14,
 'леденцы': 15,
 'бритва': 16,
 'творог': 17,
 'имбирь': 18,
 'вареники': 19,
 'семечки': 20,
 'батончики': 21,
 'пельмени': 22,
 'доширак': 23,
 'лук': 24,
 'колбасакопчёная': 25,
 'гречка': 26,
 'шоколад': 27,
 'помидоры': 28,
 'молоко': 29,
 'лапша': 30,
 'банан': 31,
 'шампунь': 32,
 'чипсы': 33,
 'вино': 34,
 'чайчёрный': 35,
 'лимонад': 36,
 'сок': 37,
 'кефир': 38,
 'огурцы': 39,
 'йогурт': 40,
 'купаты': 41,
 'хлебчёрный': 42,
 'пепсикола': 43,
 'хлеббелый': 44,
 'ряженка': 45,
 'виски': 46,
 'макароны': 47,
 'сыр': 48,
 'колбасадокторская': 49,
 'сухарики': 50,
 'говядина': 51,
 'свинина': 52,
 'ченок': 53,
 'хлебржаной': 54,
 'липтон': 55,
 'глазированныйсырок': 56,
 'кашапшённая': 57,
 'чайзелёный': 58,
 'яблоко': 59,
 'апельсины': 60,
 'бенз

In [6]:
total_length = len(df.groupby(['UserId', 'CheckId']))

max_check_items = 9

data = []

for i in range(len(df.groupby('UserId').count())):
    user_items = df[df.UserId == i]

    for j in range(max_check_items):
        check_items = user_items[user_items.CheckId == j]

        if len(check_items) == 0:
            break

        subdata = []
        for item_name in check_items.ProductName:
            subdata.append(tokenizer.word_index[item_name.lower()])

        data.append(subdata)

data = list(filter(lambda x: len(x) >= max_check_items, data))
data = list(map(lambda x: x[:max_check_items], data))

In [ ]:
def train():
    train_x = np.asarray(data, dtype='float32').reshape(-1, max_check_items)
    train_y = train_x[:, -1].reshape(-1, 1)
    train_y = tf.keras.utils.to_categorical(train_y, num_classes=len(tokenizer.word_index))
    train_x = train_x[:, :max_check_items - 1]

    inputs = tf.keras.layers.Input(shape=(max_check_items - 1,))
    dense1 = tf.keras.layers.Dense(256, activation=tf.nn.relu)(inputs)
    dense2 = tf.keras.layers.Dense(128, activation=tf.nn.relu)(dense1)
    dense3 = tf.keras.layers.Dense(64, activation=tf.nn.relu)(dense2)

    outputs = tf.keras.layers.Dense(len(tokenizer.word_index), activation=tf.nn.softmax)(dense3)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.build((-1, max_check_items - 1))

    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        metrics=['accuracy'],
    )

    history = model.fit(
        train_x,
        train_y, 
        batch_size=128,
        epochs=25,
        verbose=True,
    )

    return history


In [7]:
neighbors_length = 3

data_index = 0

nnm = NearestNeighbors(n_neighbors=neighbors_length, metric='cosine', algorithm='brute').fit(data)

distance, indices = nnm.kneighbors([data[data_index]])

neighbors = np.array(data)[indices.reshape(neighbors_length)]


In [10]:
user_id = 0

checks = df[df.UserId == user_id]

In [15]:
df

,UserId,CheckId,ProductName,ProductCost,MerchantName,MCC,token
0,2217,17,Пополнениевиртуальногосчёта,4500,STEAM,5816,76
1,635,17,Творог,40,Пятёрочка,5441,17
2,551,38,Вареники,259,Магнит,5441,19
3,2852,14,Шампунь,50,Магнит,5441,32
4,2365,8,СТРЕПСИЛССВИТАМИНОМС,163,apteka.ru,5912,142
...,...,...,...,...,...,...,...
700640,595,16,Апельсины,60,Пятёрочка,5441,60
700641,1463,33,Доширак,60,Магнит,5441,23
700642,1641,36,Мандарины,80,Пятёрочка,5441,14
700643,206,1,Пицца,350,Перекресток,5441,12
